In [15]:
import numpy as np
import pandas as pd
#from google.colab import files
import io


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

train = pd.read_csv('data/2020.AI.bike-train.csv')
test = pd.read_csv('data/2020.AI.bike-test.csv')

#train = train.fillna(train.mean())  # train 데이터의 결측치를 평균값으로 대체
#test = test.fillna(test.mean())  # test 데이터의 결측치를 평균값으로 대체

#uploaded = files.upload()
#train = pd.read_csv(io.BytesIO(uploaded['2020.train.csv']))

#uploaded = files.upload()
#test = pd.read_csv(io.BytesIO(uploaded['2020.test.csv']))

#print(train.head())


print(train.shape)
print(test.shape)

(1000, 11)
(327, 10)


In [16]:
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)
# 정답레이블 분리하고
train_x = train.drop(['count'],axis=1)
train_y =train.iloc[:,-1:]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [17]:
# 모든 모델 중 모든 지표에서 가장 낮은 에러를 보인 Extra Trees 모델 선택
# 파라미터 조금씩 조정하면 성능 더 좋아질 수도 있을텐데 일단 바로 사용할게요
# 참고로 카톡에 올렸던 더 크고 결측치 있는 데이터로 비교했을 때도 가장 성능 좋은건 Extra Trees였음
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, VotingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# 1번 모델 설정 - 이예원

# 특성 선택 및 전처리
numeric_features = ['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
                    'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
                    'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

# 모델 설정
models = [
    ('extra_trees', ExtraTreesRegressor(n_estimators=100, random_state=42)),
    ('lightgbm', LGBMRegressor(n_estimators=100, random_state=42)),
    ('voting', VotingRegressor(estimators=[
        ('extra_trees', ExtraTreesRegressor(n_estimators=100, random_state=42)),
        ('lightgbm', LGBMRegressor(n_estimators=100, random_state=42)),
        ('xgboost', XGBRegressor(n_estimators=100, random_state=42))
    ]))
]

# Stacking 모델 설정
stacking_model = StackingRegressor(
    estimators=models,
    final_estimator=LinearRegression()
)

# 전체 파이프라인
model1 = Pipeline(steps=[
    ('preprocessor', preprocessor),  # 전처리 단계
    ('regressor', stacking_model)    # 모델 학습 단계
])

In [18]:
# 2번 모델 설정 - 박수민
model2 = make_pipeline(StandardScaler(), PCA(n_components = 7), PolynomialFeatures(degree = 2, interaction_only = False, include_bias=True), Ridge(alpha =5, random_state = 1))

In [19]:
# 3번모델 설정 - 나현선
# 수치형 특성 선택
numeric_features = ['hour', 'hour_bef_temperature', 'hour_bef_precipitation', 
                    'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility', 
                    'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5']
# 전처리 파이프라인 생성
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])
# 모델 설정
models = [
    ('linear_regression', LinearRegression()),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=100, random_state=42))
]
# Stacking 모델 설정
stacking_model = StackingRegressor(
    estimators=models,
    final_estimator=LinearRegression()
)
model3 = make_pipeline(
    preprocessor,
    stacking_model
)

In [20]:
#4번모델 - 김승현
from sklearn.base import BaseEstimator, TransformerMixin
def set_visibility(x):
    if x > 0.8:
        return 2
    elif x > 0.36:
        return 1
    else:
        return 0
vec_func = np.vectorize(set_visibility)

def visi_trans(X):
    X=np.power(X,3)
    X/=np.max(X)
    return vec_func(X)

class PostProcessedPredictor(Ridge):
    def fit(self, X, y):
        super().fit(X,np.cbrt(y))
        return self
        
    def predict(self, X):
        predictions = super().predict(X)  # 기존 예측 수행
        return predictions ** 3            # 후처리: 예측 결과에 3제곱을 적용
        
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

sqrt_features = ["hour_bef_windspeed", "hour_bef_ozone", "hour_bef_pm10", "hour_bef_pm2.5"]
visi_features = ["hour_bef_visibility"]
etc_features = ["hour",'hour_bef_temperature', 'hour_bef_precipitation', 'hour_bef_humidity',]
sqrt_transformer = FunctionTransformer(np.sqrt, validate=True)
visi_transformer = FunctionTransformer(visi_trans, validate=True)

preprocessor2 = ColumnTransformer(
    transformers=[
        ('root', sqrt_transformer, sqrt_features),
        ('visi', visi_transformer, visi_features),
        ("ect", "passthrough",etc_features,)
    ])

model4 = Pipeline(steps=[
    ('preprocessor', preprocessor2),
    ('regressor', PostProcessedPredictor(0.01))
])
print(X_train.shape)
print(preprocessor2.fit_transform(X_train).shape)

(700, 9)
(700, 9)


In [21]:
#5번 모델 - 김요찬
from sklearn.neighbors import KNeighborsRegressor
model5=make_pipeline(KNeighborsRegressor())

In [22]:
#모델 할당
models_assignment = [
    ("model1 - 이예원" , model1),
    ("model2 - 박수민" , model2),
    ("model3 - 나현선", model3),
    ("model4 - 김승현", model4),
    ("model5 - 김요찬", model5)
]

In [23]:
from sklearn.ensemble import StackingRegressor
reg_stk = StackingRegressor(models_assignment)
reg_stk.fit(X_train, y_train)


# 테스트 데이터에 대한 예측 수행
Y_pred_stk = reg_stk.predict(test)

#저장 후 파일 다운로드
submission = pd.DataFrame(Y_pred_stk,columns=['count'])

submission.to_csv('sample_submission_stk.csv',index_label='id')

C:\Users\PSM\anaconda3\Lib\site-packages\sklearn\ensemble\_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 9
[LightGBM] [Info] Start training from score 110.880000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [ ]:
from sklearn.ensemble import VotingRegressor
reg_vote  = VotingRegressor(models_assignment)
reg_vote.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
Y_pred_vote = reg_vote.predict(test)

#저장 후 파일 다운로드
submission = pd.DataFrame(Y_pred_vote,columns=['count'])

submission.to_csv('sample_submission_vote.csv',index_label='id')

In [13]:
Val_Y_preds= {}

for t in models_assignment:
    t[1].fit(X_train, y_train)
    Val_Y_preds[t[0]] = t[1].predict(X_val)
    
Val_Y_preds["Assembel - stacking"] = reg_stk.predict(X_val)
Val_Y_preds["Assembel - voting"] = reg_vote.predict(X_val)

for name, y_pred in Val_Y_preds.items():
    print(f"{name} :")
    print("MAE : %.2f, MSE : %.2f, R2 : %.2f\n"%((mean_absolute_error(y_val,y_pred)), mean_squared_error(y_val,y_pred), r2_score(y_val, y_pred)))

C:\Users\PSM\anaconda3\Lib\site-packages\sklearn\ensemble\_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 9
[LightGBM] [Info] Start training from score 110.880000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

C:\Users\PSM\anaconda3\Lib\site-packages\sklearn\ensemble\_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NotFittedError: This RidgeCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [14]:
for t in models_assignment:
    print(t[1].score(X_train,y_train))

0.9990502916547391
0.6668488151922463
0.9440720528893844
0.614982227240974
0.5869671733645048
